In [1]:
import os
import json
import tensorflow as tf
import numpy as np
from pycocotools.cocoeval import COCOeval

from detection.datasets import coco, data_generator
from detection.models.detectors import faster_rcnn

In [2]:
tf.enable_eager_execution()
tf.executing_eagerly()

os.environ['CUDA_VISIBLE_DEVICES'] = '0'

In [3]:
img_mean = (123.675, 116.28, 103.53)
# img_std = (58.395, 57.12, 57.375)
img_std = (1., 1., 1.)

val_dataset = coco.CocoDataSet('./COCO2017/', 'val',
                               flip_ratio=0,
                               pad_mode='fixed',
                               mean=img_mean,
                               std=img_std,
                               scale=(800, 1344))
print(len(val_dataset))


model = faster_rcnn.FasterRCNN(
    num_classes=len(val_dataset.get_categories()))


img, img_meta, bboxes, labels = val_dataset[0]
batch_imgs = tf.Variable(np.expand_dims(img, 0))
batch_metas = tf.Variable(np.expand_dims(img_meta, 0))

_ = model((batch_imgs, batch_metas), training=False)

model.load_weights('weights/faster_rcnn.h5', by_name=True)

loading annotations into memory...
Done (t=0.58s)
creating index...
index created!
4952


In [4]:
batch_size = 1

dataset_results = []
imgIds = []
for idx in range(len(val_dataset)):
    if idx % 100 == 0:
        print(idx)

    img, img_meta, _, _ = val_dataset[idx]

    proposals = model.simple_test_rpn(img, img_meta)
    res = model.simple_test_bboxes(img, img_meta, proposals)
    
    image_id = val_dataset.img_ids[idx]
    imgIds.append(image_id)
    
    for pos in range(res['class_ids'].shape[0]):
        results = dict()
        results['score'] = float(res['scores'][pos])
        results['category_id'] = int(res['class_ids'][pos])
        y1, x1, y2, x2 = [float(num) for num in list(res['rois'][pos])]
        results['bbox'] = [x1, y1, x2 - x1 + 1, y2 - y1 + 1]
        results['image_id'] = image_id
        dataset_results.append(results)

0
100
200
300
400
500
600
700
800
900
1000
1100
1200
1300
1400
1500
1600
1700
1800
1900
2000
2100
2200
2300
2400
2500
2600
2700
2800
2900
3000
3100
3200
3300
3400
3500
3600
3700
3800
3900
4000
4100
4200
4300
4400
4500
4600
4700
4800
4900


In [5]:
with open('coco_val2017_detection_result.json', 'w') as f:
    f.write(json.dumps(dataset_results))

In [6]:
coco_dt = val_dataset.coco.loadRes('coco_val2017_detection_result.json')

Loading and preparing results...
DONE (t=0.90s)
creating index...
index created!


In [7]:
cocoEval = COCOeval(val_dataset.coco, coco_dt, 'bbox')

In [8]:
cocoEval.params.imgIds = imgIds

In [9]:
cocoEval.evaluate()
cocoEval.accumulate()
cocoEval.summarize()

Running per image evaluation...
Evaluate annotation type *bbox*
DONE (t=17.52s).
Accumulating evaluation results...
DONE (t=2.73s).
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.057
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=100 ] = 0.090
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=100 ] = 0.060
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= small | maxDets=100 ] = 0.037
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=medium | maxDets=100 ] = 0.057
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= large | maxDets=100 ] = 0.076
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=  1 ] = 0.045
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets= 10 ] = 0.070
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.072
 Average Recall     (AR) @[ IoU=0.50:0.95 | area= small | maxDets=100 ] = 0.051
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=medium | maxDets=10